<a href="https://colab.research.google.com/github/RoshanChongder/Image_Processing_And_Face_Mask_Detection/blob/main/FaceMaskDetectionTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np


# keras is free source deep learning library in python
# from this library different layer will be imported
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

Loading the CSV File 

In [ ]:
# Method to read CSV Files
def read_CSV( path ):
    try:
        file = pd.read_csv( path )
        return file
    except FileNotFoundError:
        print( "CSV File not found at " + path )
        return None
    except Exception:
        print(" Unknown error appeared ")
        return None

# reading the csv file 

data_set = read_CSV('drive/MyDrive/FaceMaskDetection/DataSet.csv') 
#data_set = read_CSV('DataSet.csv') 
print( data_set.info() ) # checking info of the data set 
print( data_set.head())  # showing the first ew line of the data set 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pixels  1303 non-null   object
 1   tag     1303 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 20.5+ KB
None
                                              pixels  tag
0  248.0 247.0 246.0 248.0 243.0 187.0 80.0 127.0...    0
1  192.0 166.0 118.0 133.0 151.0 143.0 99.0 70.0 ...    0
2  87.0 97.0 90.0 74.0 54.0 40.0 38.0 52.0 80.0 7...    0
3  250.0 240.0 207.0 240.0 78.0 50.0 87.0 61.0 57...    0
4  14.0 7.0 11.0 8.0 18.0 58.0 67.0 80.0 88.0 89....    0


Data Extraction from the CSV file 

In [ ]:
# Addition of data into list from CSV for training and public testing
def data_Addition( train_ratio ) :
    global x_train , y_train , x_test , y_test , data_set , x_validate , y_validate 
    mask_px , without_mask = [] , [] 
    
    for row_count,row in data_set.iterrows():

        value = row['pixels'].split(' ')                                         # extracting the pixels as a list
        value.pop(-1)

        try :
          
          # adding the pixels
          mask_px.append(np.array(value,'float32')) if row['tag'] == 0 else  without_mask.append(np.array(value,'float32')) 
                                               
        except:

            print(" Error occurred at row number " + str(row_count) )
            print("Data Set in that row is " + row )


    x_validate , y_validate = mask_px[:100] + without_mask[:100] , [0]*100 + [1]*100   # extracting data for validation 
    mask_px , without_mask = mask_px[100:] , without_mask[100:]                        # removing the data from data set  

    # dividing the data into ratio 
    train_mx  = int(len(mask_px)*(train_ratio/100))                               # with mask 
    train_wmx = int(len(without_mask)*(train_ratio/100))                          # with out mask 

    x_train , y_train = mask_px[:train_mx] + without_mask[:train_wmx] , [0]*train_mx+[1]*train_wmx 
    x_test  , y_test  = mask_px[train_mx:] + without_mask[train_wmx:] , [0]*len(mask_px[train_mx:])+[1]*len(without_mask[train_wmx:])

    print( len(x_train) , len(y_train) , len(x_test) , len(y_test) )      

# Now we will do training and public testing

x_train , y_train = [] , []                                                       # data the will be used for training will added in this two lists
x_test , y_test = [] , []                                                         # data that will be used for public testing will be added here
x_validate , y_validate = [] , []                                                 # for building the confusion matrix 

data_Addition( 70 )    # addition of data in the lists for training and testing

print( "Training : " , len(x_train) , "Testing : " , len(x_test) )

# checking the lists
print( x_train[:2])
print( y_train[:2])

print(x_train[0].shape )

771 771 332 332
Training :  771 Testing :  332
[array([254., 254., 253., ..., 255., 255., 255.], dtype=float32), array([253., 253., 254., ..., 235., 211., 231.], dtype=float32)]
[0, 0]
(2304,)


Convertion to numpy array 

In [ ]:
# Method to Convert from list to Numpy Arrays
def Convert_to_np_Array():
    global x_train, x_test, y_test , y_train
    # Converting list to numpy Array
    x_train = np.array(x_train, 'float32')
    y_train = np.array(y_train, 'float32')
    x_test = np.array(x_test, 'float32')
    y_test = np.array(y_test, 'float32')



# As the Keras Module only takes numpy arrays as input
# we need to convert this lists into numpy arrays


Convert_to_np_Array()
print( type(x_train) )  # checking if the type has changed to numpy or not 
print( y_train )

<class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1.

Rescale and reshape

In [ ]:
def Rescale():
    # Normalizing the data
    # why data normalization is required - https://www.import.io/post/what-is-data-normalization-and-why-is-it-important/
    # how it's work - https://www.educative.io/edpresso/data-normalization-in-python
    # read Out - https://www.mathsisfun.com/data/standard-deviation.html

    global x_train , x_test , y_test , y_train

    # we are basically rescaling
    x_train -= np.mean(x_train, axis=0)
    x_train /= np.std(x_train, axis=0)  # CENTRALIZING THE DATA

    x_test -= np.mean(x_test, axis=0)
    x_test /= np.std(x_test, axis=0)


def Reshape( width , height ):

    global x_train , y_train , x_test , y_test

    x_train = x_train.reshape(x_train.shape[0], width, height, 1)
    x_test = x_test.reshape(x_test.shape[0], width, height, 1)

    print( "Shape of x " , x_train.shape )

    # WHAT THIS FUNCTION DOES TO_CATEGORICAL
    y_train = np_utils.to_categorical(y_train,num_classes=2)
    y_test = np_utils.to_categorical(y_test,num_classes=2)
    
    
    print( ' Reshape method is called  ')

# Rescalling the data
Rescale()

# Reshaping the x train and y train in to a one d array
Reshape(48,48)

Shape of x  (771, 48, 48, 1)
 Reshape method is called  


CNN

In [ ]:

def Design_CNN():

    # The number of epochs is a hyperparameter
    # that defines the number times that the learning algorithm will work through the entire training dataset
    features =  64
    Batch_size = 128
    Label = 2
    epoch = 50
    global  x_train, y_train , x_test , y_test 
    model = Sequential()

    ## Layer 1
    # adding layers
    # Conv2d is used as the image are in 2d format
    # here we are trying extract input
    # Relu is a rectifier

    # Search Kernal size
    model.add(Conv2D(features,kernel_size=(1,1),activation='relu',input_shape=(x_train.shape[1:])))
    model.add(Conv2D(features,kernel_size=(3, 3),activation='relu'))

    # adding a max pooling 2D layer
    # It mainly helps to control over fitting
    # can use average pooling layer also
    model.add( MaxPool2D(pool_size=(2,2),strides=(2,2)) )

    # adding a drop out layer
    model.add(Dropout(0.4))

    ## 2ND layer
    model.add(Conv2D(features, kernel_size=(1, 1), activation='relu'))
    model.add(Conv2D(features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.4))

    ## 3RD Layer
    model.add(Conv2D(2*features, kernel_size=(1, 1), activation='relu'))
    model.add(Conv2D(2*features, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # ## 4TH Layer
    model.add(Conv2D(2*features, kernel_size=(1, 1), activation='relu'))
    model.add(Conv2D(2*features, kernel_size=(1, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    model.add( Flatten() )

    # adding dense layers
    model.add(Dense(2**3 * features, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2 ** 3 * features, activation='relu'))
    model.add(Dropout(0.2))

    # Adding the final layers
    model.add(Dense(Label,activation='softmax')) # Activation is softmax as we want to bind in the 7 labels of 0ptions

    model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
    model.fit(x_train,y_train,batch_size=Batch_size,epochs=epoch,verbose=1,validation_data=(x_test,y_test), shuffle=True )

    # Saving the model

    EmotionDetectJson = model.to_json()
    
    # drive/MyDrive/FaceMaskDetection/TrainedModel/July11_21/Model2/
    # drive/MyDrive/FaceMaskDetection/TrainedModel/July11_21/Model2/
    
    with open("drive/MyDrive/FaceMaskDetection/TrainedModel/July16_21/Model1/fer.json","w") as file :
        file.write(EmotionDetectJson)
    model.save_weights("drive/MyDrive/FaceMaskDetection/TrainedModel/July16_21/Model1/fer.h5")




# Start designing Our CNN

# To build the model we will be using sequential Type

Design_CNN()




Epoch 1/50
7/7 [==============================] - 2s 142ms/step - loss: 0.6429 - accuracy: 0.6397 - val_loss: 0.5881 - val_accuracy: 0.6476
Epoch 2/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5723 - accuracy: 0.7108 - val_loss: 0.5762 - val_accuracy: 0.7078
Epoch 3/50
7/7 [==============================] - 0s 38ms/step - loss: 0.4587 - accuracy: 0.8179 - val_loss: 0.4664 - val_accuracy: 0.8253
Epoch 4/50
7/7 [==============================] - 0s 37ms/step - loss: 0.3725 - accuracy: 0.8754 - val_loss: 0.4058 - val_accuracy: 0.8584
Epoch 5/50
7/7 [==============================] - 0s 37ms/step - loss: 0.3716 - accuracy: 0.8806 - val_loss: 0.5600 - val_accuracy: 0.6928
Epoch 6/50
7/7 [==============================] - 0s 36ms/step - loss: 0.4868 - accuracy: 0.7463 - val_loss: 0.4633 - val_accuracy: 0.8102
Epoch 7/50
7/7 [==============================] - 0s 37ms/step - loss: 0.3806 - accuracy: 0.8591 - val_loss: 0.4174 - val_accuracy: 0.8554
Epoch 8/50
7/7 [==========